# Deepface Face Verification test 

### Import relevant libraries:

In [1]:
# !pip install deepface
from deepface import DeepFace

import matplotlib.pyplot as plt
import pandas as pd                 
import regex as re

import os

# change plot output to white
params = {"ytick.color" : "w",
          "xtick.color" : "w",
          "axes.labelcolor" : "w",
          "axes.edgecolor" : "w"}
plt.rcParams.update(params)

result_df = pd.read_csv('./result/result_debface.csv')
benchmark_df = pd.read_csv('./data/LFW-csv/pairs.csv')

In [2]:
result_df

,Model,Dataset,CM_ACC,Precision,Recall,Total Images
0,VGG-Face,LFW,88.0,100.0,88.0,9164
1,Facenet,LFW,65.0,100.0,65.0,9164
2,OpenFace,LFW,1.0,100.0,1.0,9164
3,DeepFace,LFW,26.0,100.0,26.0,9164
4,ArcFace,LFW,83.0,100.0,83.0,9164
5,VGG-Face,LFW-original,85.0,100.0,85.0,5999
6,Facenet,LFW-original,61.0,100.0,61.0,5999
7,OpenFace,LFW-original,2.0,100.0,2.0,5999
8,DeepFace,LFW-original,27.0,100.0,27.0,5999


In [ ]:
benchmark_df.head()

#### Paramater settings:

In [82]:
# select a DATASET from data folder:
# LFW, LFW-original
DATASET = "LFW"

# select a dataset from additional-data folder:
# tzuyu, kpop
DATASET_TEST = "tzuyu"

# select a model to run:
# models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace"]
MODEL = "OpenFace"

### Preset the data path directory

In [83]:
# Test additional data directory:
# dir_path = f"./additional-data/{DATASET_TEST}"

# Test dataset directory:
dir_path = f"./data/{DATASET}"

### Function to perform test with Deepface

In [91]:
def deepface_get_accuracy(img_dir_path, model_used):

    # store the results of each verification:
    tp, tn, fp, fn = 0, 0, 0, 0
    image_files = os.listdir(img_dir_path)
    truth_images = []    
    false_images = []    
    
    # If the directory has less than 2 images there is no need to test face recognition
    if len(image_files) < 2:
        return None

    else:
        # select the first image 
        image_1 = f"{img_dir_path}/{image_files[0]}"

        # loop through the other images and verify if they are the same
        for img in image_files[1:]:
            try:
                match = re.search(f"_\d.*\.jpg", str(img))
                # Find all annotation of the files, replace <NAME> with the desired person to test
                # get image name to replace the match string with empty string
                # NAME_\d\.jpg
                # NAME_\d.*\.jpg
                NAME = img.replace(match.group(0), "")
                
            except AttributeError:
                print("IMAGE: " + image_1)
                exit(1)

            
            flag = None
            if re.search(f"{NAME}_\d.*\.jpg", str(img)):
                truth_images.append(img)
                flag = True
            else:
                false_images.append(img)    
                flag = False

            try:
                img_path = f"{img_dir_path}/{img}"
                verification = DeepFace.verify(img1_path = image_1, img2_path = img_path, model_name=model_used, enforce_detection=False)
                verification_res = verification["verified"]
                print(f"image_1: {image_1}, image: {img_path}, predicted: {verification_res}, flag:{flag}")
                
                # Truth Positive if flag and predicted are True:
                if flag and verification_res:
                    tp += 1
                    
                # Truth Negative if flag and predicted are False:
                elif flag == False and verification_res == False:
                    tn += 1

                # False Positive if flag is False and predicted is True:
                elif flag == False and verification_res == True:
                    fp += 1

                # False Negative if flag is True and predicted is False:
                elif flag == True and verification_res == False:
                    fn += 1        

            except Exception as e:
                print(e)

        # confusion matrix
        cm_acc = round((tp + tn) / (tp + tn + fp + fn), 2) * 100

        return {"cm_acc": cm_acc, "tp": tp, "tn":tn, "fp":fp, "fn":fn, "image_files": len(image_files)-1}


### Deepface Dataset testing:

In [84]:
def deepface_run_dataset(dataset, dataset_path, MODEL):

    avg_truth_pos = 0
    avg_truth_neg = 0
    avg_false_pos = 0
    avg_false_neg = 0
    total_imgs = 0
    
    iteration = 0

    people = os.listdir(dataset_path)
    for plp in people:

        img_path = f"{dataset_path}/{plp}"
        data_res = deepface_get_accuracy(img_path, MODEL)

        if data_res != None:
            avg_truth_pos += data_res["tp"]
            avg_truth_neg += data_res["tn"]
            avg_false_pos += data_res["fp"]
            avg_false_neg += data_res["fn"]
            total_imgs += data_res["image_files"]

            iteration += 1
            if iteration == 1:
                print(img_path)
                break

    # calculate confusion matrix:
    # accuracy:
    cm_acc = round((avg_truth_pos + avg_truth_neg) / (avg_truth_pos + avg_truth_neg + avg_false_pos + avg_false_neg), 2) * 100
    # precision:
    cm_pre = round( (avg_truth_pos) / (avg_truth_pos + avg_false_pos), 2) * 100
    # recall: 
    cm_rec = round( (avg_truth_pos) / (avg_truth_pos + avg_false_neg), 2) * 100


    return {"Model": MODEL, "Dataset":dataset, "CM_ACC": cm_acc, "Precision":cm_pre, "Recall":cm_rec,"Total Images": total_imgs},\
         {"TP": avg_truth_pos, "TN":avg_truth_neg, "FP":avg_false_pos, "FN":avg_false_neg}

## Face recognition Benchmark Test:
- using LFW `pairs.csv` 

In [ ]:
def deepface_benchmark(dataset, dataset_path, benchmark_df, MODEL):

    tp, tn, fp, fn = 0, 0, 0, 0
    iteration = 1
    match_case = True
    for index, row in benchmark_df.iterrows():

        if iteration > 300 and match_case:
            match_case = False
            iteration = 1
        elif iteration > 300 and not match_case:
            match_case = True
            iteration = 1

        if match_case:
            img_name = row["name"]
            img_num_1 = f'{row["imagenum1"]:04d}'
            img_num_2 = f'{int(row["imagenum2"]):04d}'

            img_path_1 = f'{dataset_path}/{img_name}/{img_name}_{img_num_1}.jpg'
            img_path_2 = f'{dataset_path}/{img_name}/{img_name}_{img_num_2}.jpg'
            iteration += 1
        
        elif match_case == False:
            img_name_a = row["name"]
            img_num_1 = f'{row["imagenum1"]:04d}'
            img_name_b = f'{row["imagenum2"]}'
            img_num_2 = f'{int(row[3]):04d}'

            img_path_1 = f'{dataset_path}/{img_name}/{img_name_a}_{img_num_1}.jpg'
            img_path_2 = f'{dataset_path}/{img_name}/{img_name_b}_{img_num_2}.jpg'
            iteration += 1    

        try:
            verification = DeepFace.verify(img1_path = img_path_1, img2_path = img_path_2, model_name=MODEL, enforce_detection=False)
            verification_res = verification["verified"]
            
            # Truth Positive if flag and predicted are True:
            if match_case and verification_res:
                tp += 1
            # Truth Negative if flag and predicted are False:
            elif match_case == False and verification_res == False:
                tn += 1

            # False Positive if flag is False and predicted is True:
            elif match_case == False and verification_res == True:
                fp += 1

            # False Negative if flag is True and predicted is False:
            elif match_case == True and verification_res == False:
                fn += 1        

        except Exception as e:
            print(e)

    # calculate confusion matrix:
    cm_acc = round((tp + tn) / (tp + tn + fp + fn), 2) * 100
    # precision:
    cm_pre = round( (tp) / (tp + fp), 2) * 100
    # recall: 
    cm_rec = round( (tp) / (tp + fn), 2) * 100


    return {"Model": MODEL, "Dataset":dataset, "CM_ACC": cm_acc, "Precision":cm_pre, "Recall":cm_rec,"Total Images": index},\
         {"TP": tp, "TN":tn, "FP":fp, "FN":fn}

### Execution of the Deepface test:

In [ ]:
# random small testing
deepface_res = deepface_get_accuracy(dir_name, MODEL)

In [85]:
# manual testing
deepface_res_dataset, cm_attr = deepface_run_dataset(DATASET, dir_path, MODEL)

1/1 [==============================] - 0s 20ms/step
image_1: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0001.jpg, image: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0002.jpg, predicted: True, flag:True
1/1 [==============================] - 0s 24ms/step
image_1: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0001.jpg, image: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0003.jpg, predicted: False, flag:True
1/1 [==============================] - 0s 21ms/step
image_1: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0001.jpg, image: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0004.jpg, predicted: False, flag:True
./data/LFW/Aaron_Peirsol


In [ ]:
# benchmark testing
deepface_benchmark_res, cm_benchmark = deepface_benchmark(DATASET, dir_path, benchmark_df, MODEL)

## Result of the Deepface test:

### Small test result:

In [ ]:
cm_acc = deepface_res["cm_acc"]
truth_pos = deepface_res["tp"]
truth_neg = deepface_res["tn"]
false_pos = deepface_res["fp"]
false_neg = deepface_res["fn"]    

total_imgs = deepface_res["image_files"]

print(f"\
Accuracy of {MODEL} is {cm_acc}% out of {total_imgs} images from {DATASET} dataset.\n\
===================\n\
Truth Positive: {truth_pos} \n\
Truth Negative: {truth_neg} \n\
False Positive: {false_pos} \n\
False Negative: {false_neg} \n\
===================")

### Manual Testing:

In [86]:
# confusion matrix accuracy:
cm_acc = deepface_res_dataset["CM_ACC"]
cm_pre = deepface_res_dataset["Precision"]
cm_rec = deepface_res_dataset["Recall"]
truth_pos = cm_attr["TP"]
truth_neg = cm_attr["TN"]
false_pos = cm_attr["FP"]
false_neg = cm_attr["FN"]

# total images
total_imgs = deepface_res_dataset["Total Images"]

print(f"\
Accuracy of {MODEL} is {cm_acc}% out of {total_imgs} images from {DATASET} dataset.\n\
The Precision is {cm_pre} and Recall is {cm_rec}\n\
===================\n\
Truth Positive: {truth_pos} \n\
Truth Negative: {truth_neg} \n\
False Positive: {false_pos} \n\
False Negative: {false_neg} \n\
===================")

Accuracy of OpenFace is 33.0% out of 3 images from LFW dataset.
The Precision is 100.0 and Recall is 33.0
Truth Positive: 1 
Truth Negative: 0 
False Positive: 0 
False Negative: 2 


### Benchmark testing:

In [ ]:
# confusion matrix accuracy:
cm_acc = deepface_benchmark_res["CM_ACC"]
cm_pre = deepface_benchmark_res["Precision"]
cm_rec = deepface_benchmark_res["Recall"]
truth_pos = cm_benchmark["TP"]
truth_neg = cm_benchmark["TN"]
false_pos = cm_benchmark["FP"]
false_neg = cm_benchmark["FN"]

# total images
total_imgs = deepface_benchmark_res["Total Images"]

print(f"\
Accuracy of {MODEL} is {cm_acc}% out of {total_imgs} images from {DATASET} dataset.\n\
The Precision is {cm_pre} and Recall is {cm_rec}\n\
===================\n\
Truth Positive: {truth_pos} \n\
Truth Negative: {truth_neg} \n\
False Positive: {false_pos} \n\
False Negative: {false_neg} \n\
===================")

# Result Output:

In [14]:
# result_df_new = pd.DataFrame([deepface_benchmark_res])
result_df_new = pd.DataFrame([deepface_res_dataset])

frames = [result_df, result_df_new]
result_df = pd.concat(frames)

result_df = result_df.reset_index()
result_df.drop(columns=['index'], inplace=True)
result_df

,Model,Dataset,CM_ACC,Precision,Recall,Total Images
0,VGG-Face,LFW,88.0,100.0,88.0,9164
1,Facenet,LFW,65.0,100.0,65.0,9164
2,OpenFace,LFW,1.0,100.0,1.0,9164
3,DeepFace,LFW,26.0,100.0,26.0,9164
4,ArcFace,LFW,83.0,100.0,83.0,9164
5,VGG-Face,LFW-original,85.0,100.0,85.0,5999
6,Facenet,LFW-original,61.0,100.0,61.0,5999
7,OpenFace,LFW-original,2.0,100.0,2.0,5999
8,DeepFace,LFW-original,27.0,100.0,27.0,5999
9,Facenet,LFW,65.0,100.0,65.0,7484


In [ ]:
# Write results to csv file
result_df.to_csv("./result/result_debface.csv", index=False)

### Etc testing

### Param settings

In [101]:
IMG_1 = f"./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0001.jpg"
IMG_2 = f"./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0002.jpg"
IMG_3 = f"./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0003.jpg"
IMG_4 = f"./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0004.jpg"

IMGS = [IMG_2, IMG_3, IMG_4]

MODEL_TEST = 'OpenFace'

In [123]:
for image in IMGS:
    verification = None
    verification = DeepFace.verify(img1_path = IMG_1, img2_path = image, model_name=MODEL_TEST, enforce_detection=False)
    verification_res = verification["verified"]
    print(f"Image_1: {IMG_1}, Image_2: {image}, predicted: {verification_res}")

1/1 [==============================] - 0s 23ms/step
Image_1: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0001.jpg, Image_2: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0002.jpg, predicted: True
1/1 [==============================] - 0s 20ms/step
Image_1: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0001.jpg, Image_2: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0003.jpg, predicted: False
1/1 [==============================] - 0s 18ms/step
Image_1: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0001.jpg, Image_2: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0004.jpg, predicted: False


1/1 [==============================] - 0s 19ms/step
Image_1: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0001.jpg, Image_2: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0002.jpg, predicted: True
1/1 [==============================] - 0s 18ms/step
Image_1: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0001.jpg, Image_2: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0003.jpg, predicted: False
1/1 [==============================] - 0s 20ms/step
Image_1: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0001.jpg, Image_2: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0004.jpg, predicted: False


In [120]:
verification = DeepFace.verify(img1_path = IMG_1, img2_path = IMG_2, model_name=MODEL_TEST, enforce_detection=False)
verification_res = verification["verified"]
print(f"Image_1: {IMG_1}, Image_2: {IMG_2}, predicted: {verification_res}")

1/1 [==============================] - 0s 20ms/step
Image_1: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0001.jpg, Image_2: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0002.jpg, predicted: True


In [121]:
verification = DeepFace.verify(img1_path = IMG_1, img2_path = IMG_3, model_name=MODEL_TEST, enforce_detection=False)
verification_res = verification["verified"]
print(f"Image_1: {IMG_1}, Image_2: {IMG_3}, predicted: {verification_res}")

1/1 [==============================] - 0s 19ms/step
Image_1: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0001.jpg, Image_2: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0003.jpg, predicted: False


In [122]:
verification = DeepFace.verify(img1_path = IMG_1, img2_path = IMG_4, model_name=MODEL_TEST, enforce_detection=False)
verification_res = verification["verified"]
print(f"Image_1: {IMG_1}, Image_2: {IMG_4}, predicted: {verification_res}")

1/1 [==============================] - 0s 18ms/step
Image_1: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0001.jpg, Image_2: ./data/LFW/Aaron_Peirsol/Aaron_Peirsol_0004.jpg, predicted: False
